In [ ]:
#https://towardsdatascience.com/trying-out-dask-dataframes-in-python-for-fast-data-analysis-in-parallel-aa960c18a915

In [1]:
import dask.dataframe as ddf
import time
import pandas as pd

In [33]:
df = ddf.read_csv('random_people.csv')

In [34]:
df.head(1)

,Unnamed: 0,name,surname,salary
0,0,Henry,Joneson,5000


In [35]:
df['bonus'] = df['salary']*.5

In [36]:
dfo = pd.read_csv('random_people.csv')

In [37]:
dfo['bonus'] = dfo['salary']*.5

In [38]:
def benchmark(function, function_name):
    start = time.time()
    function()
    end = time.time()
    print("{0} seconds for {1}".format((end - start), function_name))

In [39]:
def get_bonus(df):
    df['bonus'] = df['salary']*.5

In [40]:
def test_1():
    get_bonus(df)
def test_2():
    get_bonus(dfo)

In [41]:
benchmark(test_1, 'dask dataframe nuevo')
benchmark(test_2, 'origin dataframe viejo')

0.009985685348510742 seconds for dask dataframe nuevo
0.0010008811950683594 seconds for origin dataframe viejo


In [42]:
# df3.head(10)

,Unnamed: 0,name,surname,salary,bonus
0,0,Henry,Joneson,5000,2500.0
1,1,Albert,Goodman,10000,5000.0
2,2,William,Goodman,10000,5000.0
3,3,John,Joneson,10000,5000.0
4,4,Albert,Black,10000,5000.0
5,5,Henry,Joneson,12000,6000.0
6,6,Richard,Green,5500,2750.0
7,7,Henry,Joneson,11000,5500.0
8,8,Henry,Goodman,12000,6000.0
9,9,Albert,Joneson,11000,5500.0


In [69]:
df2 = pd.concat([dfo for _ in range(1000)])

df3 = pd.concat([df2 for _ in range(500)])
# df3 = pd.concat([df2 for _ in range(1000)])  # >> 너무 너리다

In [70]:
# dfn = ddf.from_pandas(df3, npartitions=8)
dfn = ddf.from_pandas(df3, npartitions=8)

In [71]:
# del dfn

In [72]:
def test_big_get_bonus():
    get_bonus(dfn)
def test_big_get_bonus_old():
    get_bonus(df3)

def get_big_mean():
    return dfn.salary.mean().compute()

def get_big_mean_old():
    return df3.salary.mean()

def get_big_max():
    return dfn.salary.max().compute()
def get_big_max_old():
    return df3.salary.max()

def get_big_sum():
    return dfn.salary.sum().compute()
def get_big_sum_old():
    return df3.salary.sum()

def filter_df():
    df = dfn[dfn['salary']>5000]
def filter_df_old():
    df = df3[df3['salary']>5000]
    
def run_benchmarks():
    for i,f in enumerate([test_big_get_bonus, test_big_get_bonus_old,
                          get_big_mean, get_big_mean_old,
                          get_big_max,  get_big_max_old,
                          get_big_sum,  get_big_sum_old,
                          filter_df,    filter_df_old
                         ]):
        benchmark(f, f.__name__)

In [73]:
def f(x):
    return (13*x+5)%7

def apply_random_old():
    df3['random']= df3['salary'].apply(f)
    
def apply_random():
    dfn['random']= dfn['salary'].apply(f).compute()

def value_count_test():
    dfn.salary.value_counts().compute()

def value_count_test_old():
    df3.salary.value_counts()
    

In [74]:
run_benchmarks()

0.2688913345336914 seconds for test_big_get_bonus
0.6303427219390869 seconds for test_big_get_bonus_old
6.232369661331177 seconds for get_big_mean
0.7979786396026611 seconds for get_big_mean_old
2.4355757236480713 seconds for get_big_max
0.5655012130737305 seconds for get_big_max_old
1.7692925930023193 seconds for get_big_sum
0.44411611557006836 seconds for get_big_sum_old
0.0 seconds for filter_df
5.589933156967163 seconds for filter_df_old


In [ ]:
partitons = 8 데이터 2배

0.2688913345336914 seconds for test_big_get_bonus
0.6303427219390869 seconds for test_big_get_bonus_old
6.232369661331177 seconds for get_big_mean
0.7979786396026611 seconds for get_big_mean_old
2.4355757236480713 seconds for get_big_max
0.5655012130737305 seconds for get_big_max_old
1.7692925930023193 seconds for get_big_sum
0.44411611557006836 seconds for get_big_sum_old
0.0 seconds for filter_df
5.589933156967163 seconds for filter_df_old

partitons = 4
0.003843545913696289 seconds for test_big_get_bonus
0.3029954433441162 seconds for test_big_get_bonus_old
0.973236083984375 seconds for get_big_mean
0.2517683506011963 seconds for get_big_mean_old
0.9697005748748779 seconds for get_big_max
0.20313668251037598 seconds for get_big_max_old
0.8749654293060303 seconds for get_big_sum
0.2181384563446045 seconds for get_big_sum_old
0.001999378204345703 seconds for filter_df
1.5380232334136963 seconds for filter_df_old

0.003998994827270508 seconds for test_big_get_bonus
0.289959192276001 seconds for test_big_get_bonus_old
1.644618272781372 seconds for get_big_mean
0.23600196838378906 seconds for get_big_mean_old
1.4877469539642334 seconds for get_big_max
0.19876646995544434 seconds for get_big_max_old
1.4517626762390137 seconds for get_big_sum
0.18754291534423828 seconds for get_big_sum_old
0.0 seconds for filter_df
1.3441250324249268 seconds for filter_df_old

In [ ]:
benchmark(apply_random, apply_random.__name__)
benchmark(apply_random_old, apply_random_old.__name__)
benchmark(value_count_test, value_count_test.__name__)
benchmark(value_count_test_old, value_count_test_old.__name__)

c:\programdata\psc_env\lib\site-packages\dask\dataframe\core.py:2284: UserWarning: `meta` is not specified, inferred from partial data. Please provide `meta` if the result is unexpected.
  Before: .apply(func)
  After:  .apply(func, meta={'x': 'f8', 'y': 'f8'}) for dataframe result
  or:     .apply(func, meta=('x', 'f8'))            for series result
  warnings.warn(msg)


47.574191093444824 seconds for apply_random
23.962359189987183 seconds for apply_random_old


8개 partition

16.947178840637207 seconds for apply_random
11.393437147140503 seconds for apply_random_old
2.8274338245391846 seconds for value_count_test
0.4364888668060303 seconds for value_count_test_old

In [ ]:
16개 partition

17.053419589996338 seconds for apply_random
11.048610925674438 seconds for apply_random_old
1.8896231651306152 seconds for value_count_test
0.3750638961791992 seconds for value_count_test_old

In [52]:
import os
os.cpu_count()

8

In [68]:
dfn.count().compute()

Unnamed: 0    25000000
name          25000000
surname       25000000
salary        25000000
bonus         25000000
random        25000000
dtype: int64

In [30]:
dfn.salary.value_counts().compute()

10000    8000000
12000    4000000
11000    3500000
9500     3000000
13500    2500000
5500     2500000
5000     1500000
Name: salary, dtype: int64

In [31]:
dfn.head(10)

,Unnamed: 0,name,surname,salary,bonus,random
0,0,Henry,Joneson,5000,2500.0,3
0,0,Henry,Joneson,5000,2500.0,3
0,0,Henry,Joneson,5000,2500.0,3
0,0,Henry,Joneson,5000,2500.0,3
0,0,Henry,Joneson,5000,2500.0,3
0,0,Henry,Joneson,5000,2500.0,3
0,0,Henry,Joneson,5000,2500.0,3
0,0,Henry,Joneson,5000,2500.0,3
0,0,Henry,Joneson,5000,2500.0,3
0,0,Henry,Joneson,5000,2500.0,3
